<a href="https://colab.research.google.com/github/jonaidsharif/Stress-Detection-from-Social-Media-Articles/blob/main/Stress_with_LightGBM_%26_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
from plotly import express

In [3]:
import pandas as pd

# Update the file path to point to your file in Google Drive
REDDIT = '/content/drive/MyDrive/Machine Learning/😰Stress Detection from Social Media Articles/Reddit_Combi.csv'

# Specify the columns you want to load
USECOLS = ['Body_Title', 'label']

# Load the CSV file using pandas
data = pd.read_csv(REDDIT, sep=';', usecols=USECOLS)

# Display the first few rows of the dataframe
data.head()


,Body_Title,label
0,Envy to other is swallowing me Im from develop...,1
1,Nothin outta the ordinary. Paradise. Job stres...,1
2,Almost 49 and the chasm of emptiness has never...,1
3,I’m happy again After my closest friend left m...,0
4,Is it possible to recover from such a traumati...,1


In [4]:
express.pie(data_frame=data.copy()['label'].replace({0: 'negative', 1: 'positive'}),
            names='label', color='label')

In [5]:
from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_tags

CUSTOM_FILTERS = [lambda x: x.lower(),
                  remove_stopwords,
                  strip_multiple_whitespaces,
                  strip_numeric,
                  strip_punctuation,
                  strip_short,
                  strip_tags,
                 ]

data_def = data['Body_Title'].values.tolist()

texts = [preprocess_string(s=i, filters=CUSTOM_FILTERS) for i in data_def]

dictionary = Dictionary(texts)
print(dictionary)

Dictionary<14372 unique tokens: ['afford', 'age', 'beetwen', 'better', 'big']...>


In [6]:
%env TOKENIZERS_PARALLELISM=false
!pip install --quiet keybert

env: TOKENIZERS_PARALLELISM=false
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 12.0 MB/s eta 0:00:00


In [8]:
!pip install arrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.1 MB/s eta 0:00:00


In [9]:
import pandas as pd
from arrow import now # import now function from arrow
from keybert import KeyBERT
from sklearn.feature_extraction.text import TfidfVectorizer

MAX_DF = 1.0
MIN_DF = 4
MODEL = 'all-MiniLM-L12-v2'
STOP_WORDS = 'english'
DOCS = data['Body_Title'].values.tolist()

model_start = now() # Use arrow.now() to get the current time
bert = KeyBERT(model=MODEL)
bert.max_seq_length = 512
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=STOP_WORDS, min_df=MIN_DF, max_df=MAX_DF)
document_embeddings, word_embeddings = bert.extract_embeddings(docs=DOCS, vectorizer=vectorizer)
data['embedding'] = document_embeddings.tolist()
print(f'Embedding time: {now() - model_start}') # Use arrow.now() to calculate elapsed time
print(f'1. {len(document_embeddings)} documents\n2. {len(word_embeddings)} words')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning:

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding time: 0:05:55.272182
1. 3123 documents
2. 4663 words


In [10]:
def show_info(name_model, n_scores, time, y_test, y_pred):
  print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
  print(f'{name_model} done in {time}\n')
  print(classification_report(y_true=y_test, y_pred=y_pred))

In [11]:
from numpy import mean
from numpy import std
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

lgb_model = LGBMClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(data['embedding'].apply(func=pd.Series),
                                                    data['label'],
                                                    test_size=0.25,
                                                    random_state=1,
                                                    stratify=data['label']
                                                    )
n_scores_keybert = cross_val_score(lgb_model,
                                   data['embedding'].apply(func=pd.Series),
                                   data['label'], scoring='accuracy', cv=cv,
                                   n_jobs=-1)
time_start = now()
lgb_model = lgb_model.fit(X_train,y_train)
y_pred_lgb = lgb_model.predict(X_test)
time_stop = now() - time_start

show_info('LGBM', n_scores_keybert, time_stop, y_test, y_pred_lgb)

[LightGBM] [Info] Number of positive: 2059, number of negative: 283
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012948 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 97920
[LightGBM] [Info] Number of data points in the train set: 2342, number of used features: 384
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.879163 -> initscore=1.984529
[LightGBM] [Info] Start training from score 1.984529
Accuracy: 0.936 (0.011)
LGBM done in 0:00:07.036527

              precision    recall  f1-score   support

           0       0.94      0.52      0.67        95
           1       0.94      1.00      0.97       686

    accuracy                           0.94       781
   macro avg       0.94      0.76      0.82       781
weighted avg       0.94      0.94      0.93       781



In [12]:
from xgboost import XGBClassifier

params_xgb = {
    'objective': 'binary:logistic',
    'n_estimators': 250,
    'learning_rate': 0.10,
    'colsample_bytree': 0.70,
    'max_depth': 3,
    'n_jobs': 4
}


xgb_model = XGBClassifier(**params_xgb)
n_scores_keybert = cross_val_score(xgb_model,
                                   data['embedding'].apply(func=pd.Series),
                                   data['label'], scoring='accuracy', cv=cv,
                                   n_jobs=-1)
time_start = now()
xgb_model = xgb_model.fit(X_train,y_train)
y_pred_xgb = xgb_model.predict(X_test)
time_stop = now() - time_start

show_info('XGB', n_scores_keybert, time_stop, y_test, y_pred_xgb)

Accuracy: 0.940 (0.011)
XGB done in 0:00:05.541594

              precision    recall  f1-score   support

           0       0.93      0.56      0.70        95
           1       0.94      0.99      0.97       686

    accuracy                           0.94       781
   macro avg       0.94      0.78      0.83       781
weighted avg       0.94      0.94      0.93       781

